In [20]:
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder \
    .appName("MySQL_to_Parquet") \
    .config("spark.jars", "/usr/share/java/mysql-connector-java-9.2.0.jar") \
    .getOrCreate()

In [22]:
# MySQL Connection Details
MYSQL_URL = "jdbc:mysql://localhost:3306/kjcom"
MYSQL_TABLE = "sales_order"  # Replace with table name
MYSQL_USER = "magento"
MYSQL_PASSWORD = "magento123"

In [23]:
df = spark.read \
    .format("jdbc") \
    .option("url", MYSQL_URL) \
    .option("dbtable", MYSQL_TABLE) \
    .option("user", MYSQL_USER) \
    .option("password", MYSQL_PASSWORD) \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()

In [24]:
df.show()

+---------+----------+----------+-----------+--------------------+--------------------+----------+--------+-----------+--------------------+----------------------+----------------------+----------------------+----------------+--------------------+----------------------+----------------------+----------------------+------------------------+--------------------------+-------------+----------------------+----------------------+----------------------+---------------+-----------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+------------------------+---------------------------+--------------------------+---------------+----------------------+-------------------+---------------+-----------------+-----------------+-----------------+-----------+---------------+-----------------+-----------------+-----------------+-------------------+---------------------+------------------+-------------------+-----------+----------------

In [ ]:
parquet_path = "./temp_datalake"  # Replace with your path
df.write.mode("overwrite").parquet(parquet_path)

In [26]:
from google.auth.transport.requests import Request
import google.auth
from google.cloud import bigquery

In [27]:
# 🔹 (1) Get OAuth Access Token
credentials, project = google.auth.default()
credentials.refresh(Request())  # Refresh token
access_token = credentials.token

In [31]:
PROJECT_ID = "gg-bigquery-datawarehouse"
DATASET_NAME = "Test"
TABLE_NAME = "sales_order"
BQ_TABLE = f"{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}"
pandas_df = df.toPandas()

In [32]:
client = bigquery.Client(credentials=credentials, project=project)
job = client.load_table_from_dataframe(pandas_df, BQ_TABLE)
job.result() 

LoadJob<project=gg-bigquery-datawarehouse, location=US, id=6fba2b9c-f6ac-4c99-9182-ab258d5aa3ec>

In [34]:
pandas_df.tail(10)

,entity_id,state,status,coupon_code,protect_code,shipping_description,is_virtual,store_id,customer_id,base_discount_amount,...,discount_tax_compensation_invoiced,base_discount_tax_compensation_invoiced,discount_tax_compensation_refunded,base_discount_tax_compensation_refunded,shipping_incl_tax,base_shipping_incl_tax,coupon_rule_name,gift_message_id,paypal_ipn_customer_notified,dispute_status
2,3,new,canceled,None,c736e88d9e78ae108bf21bba6610e5b6,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,10.0000,10.0000,None,NaN,0,None
3,4,processing,processing,None,9b3e08730de69107f21dd1c1c3d81198,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,5.0000,5.0000,None,NaN,0,None
4,5,processing,processing,None,6a399860286bc4bfb4b189de79ac167b,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,5.0000,5.0000,None,NaN,0,None
5,6,processing,processing,None,d7af4ec477b492d190fd4f4f04c97687,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,5.0000,5.0000,None,NaN,0,None
6,7,processing,processing,None,500e25cffe47dcea784f510b914b094a,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,5.0000,5.0000,None,NaN,0,None
7,8,processing,processing,None,7cd4ff65bb335cab125fdd74d7731ca5,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,5.0000,5.0000,None,NaN,0,None
8,14,processing,processing,None,06a3d117c0664c40bf9d29638e2791b9,Flat Rate - Fixed,0,1,NaN,0.0000,...,None,None,None,None,10.0000,10.0000,None,NaN,0,None
9,15,new,pending,None,57f85a65ff16707dca586ffaaf4d612b,Free Shipping - Free,0,1,5.0,0.0000,...,None,None,None,None,0.0000,0.0000,None,NaN,0,None
10,16,new,pending,None,78180aead707d79c5ada61a823f0963f,Free Shipping - Free,0,1,5.0,0.0000,...,None,None,None,None,0.0000,0.0000,None,NaN,0,None
11,17,new,pending,None,d54ecf3e52ada72dd3cc10de96fc7004,Free Shipping - Free,0,2,4.0,0.0000,...,None,None,None,None,0.0000,0.0000,None,NaN,0,None


In [ ]:
# spark.stop()